In [21]:
!which jupyter
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm
import re

# NER Imports
import spacy
import en_core_web_sm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/home/ubuntu/thesis_env2/bin/jupyter


In [2]:
df = pd.read_csv('./data/covid19_articles_20201231.csv')

In [3]:
# date to datetime
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369047 entries, 0 to 369046
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   author      181799 non-null  object        
 1   date        369047 non-null  datetime64[ns]
 2   domain      369047 non-null  object        
 3   title       368962 non-null  object        
 4   url         369047 non-null  object        
 5   content     369047 non-null  object        
 6   topic_area  369047 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 19.7+ MB


In [4]:
df

,author,date,domain,title,url,content,topic_area
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer
...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business


### NER

In [5]:
ner_processor = en_core_web_sm.load()

#test = ner_processor(df.iloc[0]['content'])
#pprint(set([(X.text, X.label_) for X in test.ents if X.label_ == 'ORG']))

### Pre Processing

In [10]:
# Remove text from articles
search_text = "follow us on Facebook, Twitter, and Instagram" #2423
print(len(df[df['content'].str.contains(search_text)]))
df['content'] = df['content'].str.replace(search_text, "")
search_text = "Follow Yahoo Finance on Twitter, Facebook, Instagram, Flipboard, LinkedIn, and reddit"
print(len(df[df['content'].str.contains(search_text)]))
df['content'] = df['content'].str.replace(search_text, "")
search_text = "find us on Facebook and follow us on Twitter"
print(len(df[df['content'].str.contains(search_text)]))
df['content'] = df['content'].str.replace(search_text, "")
search_text = "on Twitter, Facebook, LinkedIn, Instagram and YouTube"
print(len(df[df['content'].str.contains(search_text)]))
df['content'] = df['content'].str.replace(search_text, "")
search_text = "Twitter - Facebook - Instagram"
print(len(df[df['content'].str.contains(search_text)]))
df['content'] = df['content'].str.replace(search_text, "")
search_text = "earnings conference call"
print(len(df[df['content'].str.contains(search_text, na=False)]))
df['content'] = df['content'].str.replace(search_text, "")

2423
27
0
17
339
2260


In [11]:
# Drop long articles - cannot be processed by Spacy
print('no. of articles: ',len(df))
df = df[df['content'].apply(lambda x: len(x) <= 1000000)]
print('no. of articles after dropping long articles: ', len(df)) # drop articles where text longer than 1,000,000 characters

no. of articles:  369047
no. of articles after dropping long articles:  369045


In [11]:
row_idx = 10332
print(df.iloc[row_idx])#['title'])
print(len(df.iloc[row_idx]['content']))

author                           Tyler Clifford@_TylerTheTyler_
date                                        2020-03-02 00:00:00
domain                                                     cnbc
title         Coronavirus facts allowed Wall Street to rally...
url           https://www.cnbc.com/2020/03/02/coronavirus-fa...
content       The stock market managed to surge in the first...
topic_area                                              finance
Name: 10333, dtype: object
3311


### POS Tagging

In [3]:
# Load part processed file
df_ner = pd.read_pickle('./data/df_ner_210717.pickle')

In [16]:
#df_ner.to_pickle('./data/df_ner_210717.pickle')

In [4]:
df_ner

,author,date,domain,title,url,content,topic_area,org_names
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[BA, Boeing, CAT, CMI, Caterpillar, Cummins In..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[ADP, AKA, CTAS, Cintas, Cloud-Based Services ..."
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[Credit Suisse, FactSet, Ford, NYSE, Tesla, We..."
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[AI, CIO, Gartner, Gartner's, H&M, IBM, IBM Data]"
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[11:30pm PST, BAFTA, CDC, Chanel, Instagram, L..."
...,...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance,
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business,
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance,
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business,


In [5]:
df_ner[df_ner['org_names'] == ""]

,author,date,domain,title,url,content,topic_area,org_names
261643,NaN,2020-08-05,marketscreener,Deutsche Post : August 2020 Roadshow Presentation,https://www.marketscreener.com/quote/stock/DEU...,INVESTOR PRESENTATION Investor Relations Deuts...,business,
261644,NaN,2020-08-05,marketscreener,Cushman & Wakefield : COVID-19 Impacts on Swit...,https://www.marketscreener.com/quote/stock/CUS...,Switzerland has had very few new COVID-19 case...,business,
261645,NaN,2020-08-05,marketscreener,Western Union : Provides Access to COVID-19 St...,https://www.marketscreener.com/quote/stock/WES...,Western Union broadens its range of services f...,business,
261646,NaN,2020-08-05,marketscreener,eXp World Holdings Reports Record Second-quart...,https://www.marketscreener.com/quote/stock/EXP...,Achieves Most Profitable Quarter in Company’s ...,business,
261647,NaN,2020-08-05,marketscreener,"CVS Health : posts strong 2Q, with virus delay...",https://www.marketscreener.com/quote/stock/CVS...,CVS Health reported surprisingly strong second...,business,
...,...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance,
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business,
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance,
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business,


In [354]:
df_lab_ner.head(6)

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listed_uo_count,listing_country,listing_country_count,org_names
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[],0,0,[],0,"[Inter Milan, Jose Mourinho’s, Lopetegui, Manc..."
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[],0,0,[],0,"[Childs, Premiership Rugby, the British & Iris..."
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,[Tesco PLC],[Tesco],1,1,[United Kingdom],1,"[BBC, BRC, EU, Sharma, Sky News, Tesco, The Br..."
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,[Wynnstay Properties Plc],[Wynnstay Properties PLC],1,1,[United Kingdom],1,[Wynnstay Properties PLC]
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[],0,0,[],0,"[Conservative Party, Dominic Cummings, Ferguso..."
5919,Barbara Kollmeyer,2020-02-18,marketwatch,Apple Warning Rattles European Tech Stocks,https://www.marketwatch.com/articles/apple-war...,AFP via Getty Images European stocks came unde...,business,"[Apple Inc., Dialog Semiconductor Plc, ASM Int...","[Apple, Dialog Semiconductor, ASM Internationa...",6,6,"[United States, United Kingdom, Netherlands, U...",6,"[AFP, ASM International, Apple, HSBC, Intesa S..."


### Clean Tagged Names
- Remove leading and trailing non-letter characters e.g. ' % 2,4 ARCU Aircraft Leasing Limited' -> 'ARCU Aircraft Leasing Limited' Note: this would also drop 3M but companies with number as first letter are very rare
- Remove names that are only 1 or 2 letters
- Remove names found in org_stopwords list

In [53]:
# load company name stopwords
filepath_stopwords = './data/company_stopwords.xlsx'
org_stopwords = pd.read_excel(filepath_stopwords, sheet_name = 'all', header = None)
org_stopwords = list(set(sorted(org_stopwords[0])))
new_stopwords = ['company']
org_stopwords = org_stopwords + new_stopwords
org_stopwords = sorted([word.lower() for word in org_stopwords])# + [word.title() for word in org_stopwords])
print('number of phrases in org stopwords list: ', len(org_stopwords))
#org_stopwords

number of phrases in org stopwords list:  141


In [193]:
# Clean Tagged Names

#df_ner = pd.read_pickle('./data/df_ner.pickle')
df_ner = pd.read_pickle('./data/df_lab_ner.pickle')
df_ner_clean = df_ner.copy()
col_index = df_ner_clean.columns.get_loc('org_names')

for idx, org_names in tqdm(enumerate(df_ner['org_names'])):
    
    # remove leading and trailing non-letter characters
    org_names = set([re.sub('^[^a-zA-Z]*|[^a-zA-Z]*$','',name) for name in org_names])
    
    # remove names that are only 1 or 2 or 3 letters
    org_names = set([name.lower() for name in org_names if len(name) > 3])
    #org_names = set([name for name in org_names if len(name) > 2])
    
    # exclude names in org_stopwords list - difference between 2 sets
    org_names = org_names - set(org_stopwords) 
    df_ner_clean.iat[idx, col_index] = sorted(org_names)
    
# Save df after POS tagging
df_ner_clean.to_pickle('./data/df_ner_clean.pickle')

1000it [00:00, 11474.11it/s]


In [194]:
df_ner_clean = pd.read_pickle('./data/df_ner_clean.pickle')
df_ner_clean.head(3)

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listed_uo_count,listing_country,listing_country_count,org_names
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[],0,0,[],0,"[inter milan, jose mourinho’s, lopetegui, roma..."
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[],0,0,[],0,"[childs, premiership rugby, the british & iris..."
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,[Tesco PLC],[Tesco],1,1,[United Kingdom],1,"[sharma, tesco]"


In [195]:
import itertools 

# List of all org names
ner_org_names = sorted(set(list(itertools.chain(*[names for names in df_ner_clean['org_names']]))))
df_ner_org_names = pd.DataFrame(ner_org_names)
df_ner_org_names.to_pickle('./data/df_ner_org_names.pickle')
len(ner_org_names)

7815

In [196]:
#df_ner[df_ner['org_names'].apply(lambda x: 'guaranty agreement' in x)]
#df_ner_clean[df_ner_clean['org_names'].apply(lambda x: 'a\ncredit and guaranty agreement' in x)]

In [197]:
#ner_org_names[::-1]
ner_org_names[:5]

['a (excellent',
 'a biologics license application (bla',
 'a chartered accountant',
 'a comissão de auditoria manuel ramos de sousa sebastião estela de magalhães',
 'a department of health and social care (dhsc']

### Checking similarity of NER word against company names in company list

Rule 1 
- When the NER is only 1 word  
- The comparison is reduced to the set of common words  
- When the similarity ratio is higher than 90% it is accepted

Problems
- When a one word NER is found in multiple company names, e.g Southwest (found NER) returns company names 'Southwest Airlines' and 'Southwest Gas Corp' and 'Southwest Capital'

Rule 2
- When the NER is more than 1 word
- When part of the NER exists in the stop words list e.g. 'Capital One Financial Corp', financial and corp are words in the stop words list
- When the similarity ratio is higher than 95% it is accepted

Rule 3
- Remaining words
- Similarity ratio on best matching substring being 90% and higher


Potential Rule
- When a name is only 1 word only take the match which is the shortest length. E.g Express -> matches Amercian Express and Express Inc -> it should only take Express Inc

In [198]:
# Import listed companies
filepath_companies = './data/company_names_listed.xlsx'
companies_list = pd.read_excel(filepath_companies, sheet_name = 'company_names')

# Drop companies names that are 2 or 3 characters long
companies_list = companies_list.drop(labels = companies_list[companies_list['name'].str.len() < 4].index)

# Countries of interest
countries_included = ['United States', 'Canada', 'Australia', 'United Kingdom']
companies_list = companies_list[companies_list['country'].isin(countries_included)]

companies_list['country'].value_counts()

United States     8848
Canada            3405
Australia         1873
United Kingdom    1424
Name: country, dtype: int64

In [199]:
companies_list

,id,name,common_names,ticker_symbol,country,industry,subindustry
0,35931,"The Ultimate Software Group, Inc.",NaN,ULTI,United States,Information Technology,Software
1,35908,"U.S. Personnel, Inc.",NaN,USPE,United States,Industrials,Professional Services
2,203152,e.Digital Corporation,NaN,EDIG,United States,Information Technology,"Technology Hardware, Storage and Peripherals"
3,300794,Rowan Companies plc,NaN,RDC,United States,Energy,Energy Equipment and Services
5,388503,"HopFed Bancorp, Inc.",NaN,HFBC,United States,Financials,Thrifts and Mortgage Finance
...,...,...,...,...,...,...,...
51259,new003,"Revolution Medicines, Inc.",NaN,RVMD,United States,Healthcare,Biotechnology
51260,new004,PG&E Corporation,Pacific Gas and Electric Co,PCG,United States,Utilities,Multi-Utilities
51261,new005,"Virgin Galactic Holdings, Inc.",NaN,SPCE,United States,Industrials,Aerospace and Defense
51262,new006,"Rocket Companies, Inc.",NaN,RKT,United States,Financials,Thrifts and Mortgage Finance


In [200]:
# most common words in listed company names
#companies_list['name'].values

from collections import Counter
results = Counter()
companies_list['name'].str.lower().str.split().apply(results.update)
companies_list_common_words = sorted([k for k, v in results.items() if v > 99])
#print(results)

In [201]:
#print(results)

In [202]:
print(companies_list_common_words)

['&', 'american', 'and', 'bancorp,', 'bank', 'capital', 'company', 'corp.', 'corporation', 'energy', 'energy,', 'exploration', 'financial', 'first', 'global', 'gold', 'group', 'group,', 'holdings', 'holdings,', 'inc.', 'incorporated', 'international', 'international,', 'investment', 'limited', 'ltd', 'ltd.', 'metals', 'minerals', 'mining', 'of', 'oil', 'pharmaceuticals,', 'plc', 'resources', 'services', 'systems,', 'technologies', 'technologies,', 'technology', 'the', 'therapeutics,', 'trust', 'ventures']


In [203]:
df_ner_clean

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listed_uo_count,listing_country,listing_country_count,org_names
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[],0,0,[],0,"[inter milan, jose mourinho’s, lopetegui, roma..."
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[],0,0,[],0,"[childs, premiership rugby, the british & iris..."
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,[Tesco PLC],[Tesco],1,1,[United Kingdom],1,"[sharma, tesco]"
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,[Wynnstay Properties Plc],[Wynnstay Properties PLC],1,1,[United Kingdom],1,[wynnstay properties plc]
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[],0,0,[],0,"[conservative party, dominic cummings, ferguso..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169245,By Reuters,2020-06-24,nytimes,"With No Tourists to Watch Migration, Kenyan Op...",https://www.nytimes.com/reuters/2020/06/24/bus...,NAIROBI — For Kenyan-based safari operator Saf...,business,[],[],0,0,[],0,"[chaumont, maasai mara, nairobi, nairobi tente..."
306948,NaN,2020-09-12,marketscreener,"Pfizer, BioNTech propose expanding COVID-19 va...",https://www.marketscreener.com/news/latest/Pfi...,Pfizer Inc and BioNTech SE\non Saturday propos...,business,[Pfizer Inc.],[Pfizer Inc],1,1,[United States],1,[the u.s food and drug\nadministration]
111840,NaN,2020-05-13,marketscreener,Ameresco : Announces Commercial Operations of ...,https://www.marketscreener.com/AMERESCO-INC-64...,Beale Hill Wind Farm is the first renewable ge...,business,"[Ameresco, Inc.]","[Ameresco, Inc.]",1,1,[United States],1,"[about ameresco, ameresco, ameresco executive,..."
72139,NaN,2020-04-16,marketscreener,"L'Oréal: News release: ""First Quarter 2020 Sales""",https://www.marketscreener.com/quote/stock/L-O...,"Clichy, 16 April 2020 at 6:00 p.m. First Quart...",business,[L'Oréal S.A.],[L'Oréal],1,1,[France],1,"[appendix, autorit, banco alimentare, columbia..."


In [204]:
# Drop countries not interested in from labelled file
col_index_lc = df_ner_clean.columns.get_loc('listing_country')
col_index_onl = df_ner_clean.columns.get_loc('org_names_listed')
col_index_onluo = df_ner_clean.columns.get_loc('org_names_listed_uo')
col_index_lc2 = df_ner_clean.columns.get_loc('listed_count')

for idx in range(len(df_ner_clean)):
    org_names_listed = []
    org_names_listed_uo = []
    countries_to_add = []

    for idx_c, country in enumerate(df_ner_clean.iloc[idx, col_index_lc]):
        if country in countries_included:
            listed_name = df_ner_clean.iat[idx, col_index_onl][idx_c]
            listed_name_uo = df_ner_clean.iat[idx, col_index_onluo][idx_c]
            org_names_listed.append(listed_name)
            org_names_listed_uo.append(listed_name_uo)
            countries_to_add.append(country)
            
    df_ner_clean.iat[idx, col_index_onl] = org_names_listed
    df_ner_clean.iat[idx, col_index_onluo] = org_names_listed_uo
    df_ner_clean.iat[idx, col_index_lc] = countries_to_add
    df_ner_clean.iat[idx, col_index_lc2] = len(org_names_listed)

df_ner_clean = df_ner_clean.drop(['listed_uo_count', 'listing_country_count'], axis=1)

In [214]:
df_ner_clean['filtered_names'] = ''
df_ner_clean['filtered_names_match'] = ''
col_index_fn = df_ner_clean.columns.get_loc('filtered_names')
col_index_fnm = df_ner_clean.columns.get_loc('filtered_names_match')
#print_on = True
print_on = False

for idx, names in tqdm(enumerate(df_ner_clean['org_names'])):
#for idx, names in enumerate(df_ner_clean['org_names'][800:850]):
    if print_on:
        print('\n', idx, names, 'labels: ', df_ner_clean.iat[idx,col_index_onl])
    filtered_names = []
    filtered_names_match = []
    for name in names:
        
        # Rule 1 - NER name only 1 word -> set of common words  -> similarity ratio on set
        if len(name.split(' ')) == 1:
            #processor = lambda x: set(x.lower().split()) - set(companies_list_common_words)
            processor_2 = lambda x: x.lower()
            best_match = process.extractOne(query=name, choices=companies_list['name'], processor=processor, scorer=fuzz.token_sort_ratio, score_cutoff=98) # 0.65 128/66/73
            #best_match = process.extractOne(query=name, choices=companies_list['name'], processor=processor_2, scorer=fuzz.ratio, score_cutoff=75)
            if best_match:
                if print_on:
                    print('rule1: ', name, '-', best_match, best_match[1])
                filtered_names.append(name)
                filtered_names_match.append(best_match[0])            
                    
        # Rule 2 - NER name longer than 1 word plus part of name in stopwords list -> similarity ratio on set
        elif set(name.split(' ')).intersection(org_stopwords):
            processor_2 = lambda x: x.lower()
            best_match = process.extractOne(query=name, choices=companies_list['name'], processor=processor_2, scorer=fuzz.token_set_ratio, score_cutoff=95) # 0.65 128/66/73
            if best_match:
                if print_on:
                    print('rule2: ', name, '-', best_match[0], best_match[1])
                filtered_names.append(name)
                filtered_names_match.append(best_match[0])

        else:
            best_match = process.extractOne(query=name, choices=companies_list['name'], scorer=fuzz.partial_ratio, score_cutoff=96)
            if best_match:
                if print_on:
                    print(f'best match: {best_match}')
                    print('rule3: ', name, '-', best_match[0], best_match[1])
                filtered_names.append(name)
                filtered_names_match.append(best_match[0])
                    
    df_ner_clean.iat[idx,col_index_fn] = list(set(filtered_names))
    df_ner_clean.iat[idx,col_index_fnm] = list(set(filtered_names_match))

54it [01:18,  1.20s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'the
company']
59it [01:37,  4.68s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'global']
428it [13:49,  1.15s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'the
company']
533it [17:13,  1.61s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'the
company']
696it [23:24,  1.03s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'the
company']
742it [25:43,  5.74s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'investment']
772it [26:47,  1.69s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons 

In [217]:
df_ner_clean['count_matches'] = ''
df_ner_clean['count_lab_not_found'] = ''
df_ner_clean['count_found_not_lab'] = ''
col_index_onm = df_ner_clean.columns.get_loc('org_names_listed')
col_index_fnm = df_ner_clean.columns.get_loc('filtered_names_match')
col_index_cm = df_ner_clean.columns.get_loc('count_matches')
col_index_clnf = df_ner_clean.columns.get_loc('count_lab_not_found')
col_index_cfnl = df_ner_clean.columns.get_loc('count_found_not_lab')

for idx in range(len(df_ner_clean)):
    # True Positives
    count_matches = len(list(set(df_ner_clean.iloc[idx,col_index_onm]).intersection(df_ner_clean.iloc[idx,col_index_fnm])))
    # False Negatives
    count_lab_not_found = len(list(set(df_ner_clean.iloc[idx,col_index_onm]).difference(df_ner_clean.iloc[idx,col_index_fnm]))) # labelled but not found in NER
    # False Positives
    count_found_not_lab = len(list(set(df_ner_clean.iloc[idx,col_index_fnm]).difference(df_ner_clean.iloc[idx,col_index_onm]))) # found in NER but not labelled
    
    df_ner_clean.iat[idx, col_index_cm] = count_matches
    df_ner_clean.iat[idx, col_index_clnf] = count_lab_not_found
    df_ner_clean.iat[idx, col_index_cfnl] = count_found_not_lab

In [218]:
# Calculate performance

true_pos = sum(df_ner_clean['count_matches'])
false_neg = sum(df_ner_clean['count_lab_not_found'])
false_pos = sum(df_ner_clean['count_found_not_lab'])

print(true_pos, false_neg, false_pos)

precision = true_pos / (true_pos + false_pos) # low number means lots of false positives
recall = true_pos / (true_pos + false_neg) # low number means lots of false negatives
f1_score = 2 * (precision * recall) / (precision + recall)
print(f'precision = {round(precision,2)}')
print(f'recall = {round(recall,2)}')
print(f'f1 score = {round(f1_score,2)}')

547 281 339
precision = 0.62
recall = 0.66
f1 score = 0.64


In [164]:
df_ner_clean[30:50]#.head(20)

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listing_country,org_names,filtered_names,filtered_names_match,count_matches,count_lab_not_found,count_found_not_lab
313847,NaN,2020-09-16,marketscreener,PASSUR Aerospace : Announces Changes to the Da...,https://www.marketscreener.com/quote/stock/PAS...,"PASSUR Aerospace, Inc. (OTC: PSSR) ('PASSUR' o...",business,"[PASSUR Aerospace, Inc.]",[PASSUR],1,[United States],"[about passur aerospace, eastern, important, i...","[eastern, inc. passur aerospace, passur aerosp...","[PASSUR Aerospace, Inc., The Eastern Company]",1,0,1
156620,By Sarah Bahr,2020-06-16,nytimes,‘Bold and the Beautiful’ to Resume Production ...,https://www.nytimes.com/2020/06/16/arts/televi...,The Forresters will be back scheming and sasha...,business,[],[],0,[],"[bell-phillip tv productions, black entertainm...",[],[],0,0,0
36890,Patricia Riddell,2020-03-25,theconversation,Why coronavirus lockdown rules will not be obe...,https://theconversation.com/why-coronavirus-lo...,You may have noticed that some people have res...,business,[],[],0,[],[],[],[],0,0,0
340034,NaN,2020-11-06,marketscreener,"Consumer Cos Down Slightly After Jobs Data, Pr...",https://www.marketscreener.com/news/latest/Con...,Shares of retailers and other consumer compani...,business,[The Wendy's Company],[Wendys],1,[United States],"[flynn restaurant group, npc international, pi...",[wendy's],[The Wendy's Company],1,0,0
163144,Richard Percival,2020-06-19,express,Nicola Sturgeon to ‘copycat’ EU laws in bid to...,https://www.express.co.uk/news/politics/129832...,We will use your email address only for sendi...,general,[],[],0,[],"[cabinet, guy verhofstadt, nicola sturgeon, ni...",[guy verhofstadt],[RH],0,0,1
360941,https://www.businessoffashion.com/articles/aut...,2020-12-15,businessoffashion,The BoF Podcast: A Covid Survivor’s Story,https://www.businessoffashion.com/podcasts/new...,In partnership with Podcasts In partnership wi...,consumer,[],[],0,[],"[sophia neophitou, the business of fashion]",[],[],0,0,0
55550,NaN,2020-04-04,marketscreener,U.S. dairy farmers dump milk,https://www.marketscreener.com/MILK-4377638/ne...,"It seems so wasteful, but some American dairy ...",business,[],[],0,[],[mueller dairy farm],[],[],0,0,0
347657,Charlotte Manning,2020-11-21,express,Charlie Stayt shuts down Andy Burnham after be...,https://www.express.co.uk/showbiz/tv-radio/136...,We will use your email address only for sendi...,general,[],[],0,[],"[bbc breakfast, bbc one, munchetty, stayt’s, v...",[],[],0,0,0
47979,Ed Browne,2020-03-31,express,Australia coronavirus: Australia's tough new m...,https://www.express.co.uk/news/world/1262620/A...,AUSTRALIA imposed new measures to slow the s...,general,[],[],0,[],"[jobkeeper, johns hopkins university, the nort...",[],[],0,0,0
182837,NaN,2020-07-01,finance.yahoo,Maha: Nashik records 18 new COVID-19 cases; ta...,https://in.finance.yahoo.com/news/maha-nashik-...,"Nashik, Jul 1 (PTI) As many as 18 fresh cases ...",business,[],[],0,[],"[maharashtra, malegaon]",[],[],0,0,0


In [49]:
df_ner_clean['filtered_names'] = ''
df_ner_clean['filtered_names_match'] = ''
col_index_fn = df_ner_clean.columns.get_loc('filtered_names')
col_index_fnm = df_ner_clean.columns.get_loc('filtered_names_match')

for idx, names in tqdm(enumerate(df_ner_clean['org_names'][:50])):
#for idx, names in enumerate(df_ner_clean['org_names'][800:850]):
    print('\n', idx)
    filtered_names = []
    filtered_names_match = []
    for name in names:
        for list_name in companies_list['name']:
            
            # Rule 1 - NER name only 1 word -> set of common words  -> similarity ratio on set
            if len(name.split(' ')) == 1:
                ratio = fuzz.token_set_ratio(name, list_name.lower())
                if ratio >= 90:
                    print('rule1: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name.lower())
                    
            # Rule 2 - NER name longer than 1 word plus part of name in stopwords list -> similarity ratio on set
            elif set(name.split(' ')).intersection(org_stopwords):
                ratio = fuzz.token_set_ratio(name, list_name.lower())
                if ratio >= 95:
                    print('rule2: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
                    
            else:
                ratio = fuzz.partial_ratio(name, list_name.lower())
                if ratio >= 90:
                    print('rule3: ', name, '-', list_name, ratio)
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
                    
    df_ner_clean.iat[idx,col_index_fn] = set(filtered_names)
    df_ner_clean.iat[idx,col_index_fnm] = set(filtered_names_match)

0it [00:00, ?it/s]


 0
rule1:  boeing - The Boeing Company 100
rule1:  cat - CAT Strategic Metals Corporation 100
rule1:  cat - Black Cat Syndicate Limited 100
rule1:  cat - Red Cat Holdings, Inc. 100
rule1:  caterpillar - Caterpillar Inc. 100
rule2:  cummins inc - Cummins Inc. 100
rule3:  deere & company - Deere & Company 100
rule1:  honeywell - Honeywell International Inc. 100
rule3:  the key to growth deere & company - Deere & Company 100


1it [00:03,  3.78s/it]


 1
rule1:  cintas - Cintas Corporation 100


2it [00:05,  2.84s/it]

...

47it [01:34,  1.34s/it]


 47

 48


49it [01:34,  1.22it/s]


 49


50it [01:37,  1.95s/it]


In [198]:
# Run for whole data set - Takes ~5 hours per 10,000 articles

df_ner_clean['filtered_names'] = ''
df_ner_clean['filtered_names_match'] = ''
for idx, names in tqdm(enumerate(df_ner_clean['org_names'])):
    filtered_names = []
    filtered_names_match = []
    for name in names:
        for list_name in companies_list:
            # Rule 1 - NER name only 1 word -> set of common words  -> similarity ratio on set
            if len(name.split(' ')) == 1:
                ratio = fuzz.token_set_ratio(name, list_name.lower())
                if ratio >= 90:
                    filtered_names.append(name)
                    filtered_names_match.append(list_name.lower())
            # Rule 2 - NER name longer than 1 word plus part of name in stopwords list -> similarity ratio on set
            elif set(name.split(' ')).intersection(org_stopwords):
                ratio = fuzz.token_set_ratio(name, list_name.lower())
                if ratio >= 95:
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
            else:
                ratio = fuzz.partial_ratio(name, list_name.lower())
                if ratio >= 90:
                    filtered_names.append(name)
                    filtered_names_match.append(list_name)
    if idx % 10000 == 0:
        print(f'articles to {idx} saved to file')
        df_ner_clean.to_pickle('./Data/df_ner_clean.pickle')
    df_ner_clean.at[idx,'filtered_names'] = set(filtered_names)
    df_ner_clean.at[idx,'filtered_names_match'] = set(filtered_names_match)
    
df_ner_clean.to_pickle('./Data/df_ner_clean.pickle')

In [15]:
df_ner_clean

,author,date,domain,title,url,content,topic_area,org_names
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[a dividend aristocrat for capital gains, a do..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[adp, ccc, cintas, cloud-based services for em..."
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[credit suisse, ford, musk, nyse, tsla, wedbus..."
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[cio dive, fedex, gartner, gartner's cio, h&m,..."
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[cdc, chanel, climate brief, moschino, new sta..."
...,...,...,...,...,...,...,...,...
48778,Jan Lopatka,2020-03-31,reuters,"Czech virus chief defends face masks order, sa...",https://www.reuters.com/article/health-coronav...,- The Czech government’s coronavirus tsar on ...,business,"[health ministry, the world health organisatio..."
48779,Francois Murphy,2020-03-31,reuters,UPDATE 1-Austrian economy to shrink 3.2% in 'm...,https://www.reuters.com/article/austria-econom...,"By Francois Murphy VIENNA, March 31 (Reuters)...",business,"[ecb, holzmann, onb, reuters, the austrian nat..."
48780,Francois Murphy,2020-03-31,reuters,Austrian economy to shrink 3.2% in 'moderate' ...,https://www.reuters.com/article/uk-austria-eco...,Austria’s economy will shrink 3.2% this year i...,business,"[ecb, holzmann, onb, the austrian national ban..."
48781,"Michelle Conlin, Linda So, Brad Heath, Grant S...",2020-03-31,reuters,UPDATE 1-INSIGHT-Coronavirus hits hundreds of ...,https://www.reuters.com/article/health-coronav...,(Updates to include latest available figures f...,business,"[nypd, reuters, shotspotter, the dallas police..."


In [4]:
df_ner_clean = pd.read_pickle('./Data/df_ner_clean.pickle')

In [5]:
df_ner_clean

,author,date,domain,title,url,content,topic_area,org_names,filtered_names,filtered_names_match
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[a dividend aristocrat for capital gains, a do...","{cummins inc, the move cummins inc, boeing, th...","{Cummins Inc, boeing company, cummins inc, Dee..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[adp, ccc, cintas, cloud-based services for em...","{manpower, inc, cintas, manpower}","{manpower inc., Manpower Inc, cintas corp., ci..."
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[credit suisse, ford, musk, nyse, tsla, wedbus...","{credit suisse, ford}","{Credit Suisse Group, ford mtr co del [f/pb], ..."
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[cio dive, fedex, gartner, gartner's cio, h&m,...","{fedex, gartner}","{fedex corp, gartner inc, fedex corporation}"
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[cdc, chanel, climate brief, moschino, new sta...",{},{}
...,...,...,...,...,...,...,...,...,...,...
48778,Jan Lopatka,2020-03-31,reuters,"Czech virus chief defends face masks order, sa...",https://www.reuters.com/article/health-coronav...,- The Czech government’s coronavirus tsar on ...,business,"[health ministry, the world health organisatio...",,
48779,Francois Murphy,2020-03-31,reuters,UPDATE 1-Austrian economy to shrink 3.2% in 'm...,https://www.reuters.com/article/austria-econom...,"By Francois Murphy VIENNA, March 31 (Reuters)...",business,"[ecb, holzmann, onb, reuters, the austrian nat...",,
48780,Francois Murphy,2020-03-31,reuters,Austrian economy to shrink 3.2% in 'moderate' ...,https://www.reuters.com/article/uk-austria-eco...,Austria’s economy will shrink 3.2% this year i...,business,"[ecb, holzmann, onb, the austrian national ban...",,
48781,"Michelle Conlin, Linda So, Brad Heath, Grant S...",2020-03-31,reuters,UPDATE 1-INSIGHT-Coronavirus hits hundreds of ...,https://www.reuters.com/article/health-coronav...,(Updates to include latest available figures f...,business,"[nypd, reuters, shotspotter, the dallas police...",,


In [102]:
print(df_ner_clean.iloc[6000][['content']].values[0])

Half of Samsung's smartphones are now made in Vietnam, where the coronavirus that has crippled the China operations of Apple and many other firms has so far had only a limited impact on its production. Apple said on Monday it would not meet its revenue guidance for the March quarter due to the coronavirus impact on both production and sales in China, where most iPhones are made. Chinese smartphone maker Xiaomi Corp last week also flagged a hit to its March quarter sales. Huawei, another major Samsung rival, has not announced any production problems, but analysts expect it will also be hit hard due to its heavy reliance on Chinese manufacturing and parts. Many Chinese and foreign firms have begun to re-open China factories that were idled for weeks, but shortages of workers and other problems have in many cases kept output to a minimum. Samsung has also largely ceded the China market to its rivals in recent years, meaning it won't suffer from the store closures and drop in demand that i

In [100]:
print(df_ner_clean.iloc[6000][['filtered_names_match']].values[0])

{'Samsung Electronics CO L', 'apple computer, inc.', 'sealand capital galaxy limited', 'apple hospitality reit inc', 'Samsung Electronics Co L', 'apple inc', 'apple inc.', 'samsung electronics co l'}
